In [1]:
import pandas as pd
import scipy as sc
import numpy as np
import sklearn
import pickle
import pathlib as Path
import matplotlib.pyplot as plt
import japanize_matplotlib
import seaborn as sb
sb.set(font='IPAexGothic')

import multiprocessing
import itertools
import collections
import datetime
import gc

from tqdm._tqdm_notebook import tqdm

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.set_option('display.max_columns', 50)

data_path = Path.Path("../data")
result_path = Path.Path("../result")

prefix = 'ana102'

In [2]:
pd.set_option('display.max_rows', 100)

# データ抽出
> ・全体データ（ana005_all_df.pkl）

In [3]:
all_df = pd.read_pickle(data_path / "ana005_all_df.pkl")
all_df

,id,B,O,S,b1,b2,b3,ballPositionLabel,ballX,ballY,batter,batterHand,battingType,dir,dist,flg_train,gameID,inning,isOuts,pitchType,pitcher,pitcherHand,speed,totalPitchingCount,y,topTeam,bottomTeam,batterTeam,pitcherTeam,is_hit0,is_hit1,is_hit2,is_hit3,is_hit4
0,0,0,0,0,False,False,False,内角低め,17.0,J,ピレラ,R,NaN,NaN,NaN,1,20202173,1回表,NaN,ストレート,今永 昇太,L,149km/h,1,0.0,広島,DeNA,広島,DeNA,False,False,False,False,False
1,1,1,0,0,False,False,False,内角低め,14.0,I,ピレラ,R,NaN,NaN,NaN,1,20202173,1回表,NaN,ストレート,今永 昇太,L,149km/h,2,1.0,広島,DeNA,広島,DeNA,False,False,False,False,False
2,2,1,0,1,False,False,False,外角高め,8.0,D,ピレラ,R,NaN,NaN,NaN,1,20202173,1回表,NaN,チェンジアップ,今永 昇太,L,137km/h,3,0.0,広島,DeNA,広島,DeNA,False,False,False,False,False
3,3,2,0,1,False,False,False,内角中心,21.0,G,ピレラ,R,NaN,NaN,NaN,1,20202173,1回表,NaN,スライダー,今永 昇太,L,138km/h,4,2.0,広島,DeNA,広島,DeNA,False,False,False,False,False
4,4,2,0,2,False,False,False,外角中心,7.0,F,ピレラ,R,G,S,38.3,1,20202173,1回表,False,チェンジアップ,今永 昇太,L,136km/h,5,4.0,広島,DeNA,広島,DeNA,True,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54203,33803,0,0,1,False,False,False,NaN,NaN,NaN,坂口 智隆,L,NaN,NaN,NaN,0,20202023,5回表,NaN,NaN,床田 寛樹,L,NaN,2,NaN,ヤクルト,広島,ヤクルト,広島,False,False,False,False,False
54204,33804,0,0,0,False,False,False,NaN,NaN,NaN,メヒア,R,NaN,NaN,NaN,0,20202640,9回表,NaN,NaN,堀岡 隼人,R,NaN,1,NaN,広島,巨人,広島,巨人,False,False,False,False,False
54205,33805,0,0,0,True,False,False,NaN,NaN,NaN,鈴木 誠也,R,NaN,NaN,NaN,0,20202864,7回裏,NaN,NaN,ディプラン,R,NaN,1,NaN,巨人,広島,広島,巨人,False,False,False,False,False
54206,33806,3,1,1,False,True,False,NaN,NaN,NaN,周東 佑京,L,NaN,NaN,NaN,0,20202806,8回裏,NaN,NaN,田村 伊知郎,R,NaN,5,NaN,西武,ソフトバンク,ソフトバンク,西武,False,False,False,False,False


# 必要な列定義
> ・id <br>
> ・ピッチェー名（pitcher）<br>
> ・ヒット以上有無（is_hit）<br>
> ・試合ID, イニング（gameID, inning） <br>
> ・バッター利き手（batterHand）<br>
> ・バッターチーム名（batterTeam）<br>
> ・出塁状態（b1, b2, b3）<br>
> ・カウント（B, O, S）<br>

In [4]:
# 必要な列定義
cols = ['id', 'pitcher', 'pitcherTeam', 'gameID', 'inning', 'batterHand', 'batterTeam', 'b1', 'b2', 'b3', 
        'B', 'O', 'S',
        'is_hit0', 'is_hit1', 'is_hit2', 'is_hit3', 'is_hit4',
        'flg_train'
       ]

In [5]:
# 使用する列のみ抽出
_all_df = all_df[cols]
_all_df

,id,pitcher,pitcherTeam,gameID,inning,batterHand,batterTeam,b1,b2,b3,B,O,S,is_hit0,is_hit1,is_hit2,is_hit3,is_hit4,flg_train
0,0,今永 昇太,DeNA,20202173,1回表,R,広島,False,False,False,0,0,0,False,False,False,False,False,1
1,1,今永 昇太,DeNA,20202173,1回表,R,広島,False,False,False,1,0,0,False,False,False,False,False,1
2,2,今永 昇太,DeNA,20202173,1回表,R,広島,False,False,False,1,0,1,False,False,False,False,False,1
3,3,今永 昇太,DeNA,20202173,1回表,R,広島,False,False,False,2,0,1,False,False,False,False,False,1
4,4,今永 昇太,DeNA,20202173,1回表,R,広島,False,False,False,2,0,2,True,True,False,False,False,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54203,33803,床田 寛樹,広島,20202023,5回表,L,ヤクルト,False,False,False,0,0,1,False,False,False,False,False,0
54204,33804,堀岡 隼人,巨人,20202640,9回表,R,広島,False,False,False,0,0,0,False,False,False,False,False,0
54205,33805,ディプラン,巨人,20202864,7回裏,R,広島,True,False,False,0,0,0,False,False,False,False,False,0
54206,33806,田村 伊知郎,西武,20202806,8回裏,L,ソフトバンク,False,True,False,3,1,1,False,False,False,False,False,0


In [6]:
# 訓練データ抽出
train_df = _all_df[_all_df['flg_train'] == 1].drop('flg_train', axis=1).copy()
train_df

,id,pitcher,pitcherTeam,gameID,inning,batterHand,batterTeam,b1,b2,b3,B,O,S,is_hit0,is_hit1,is_hit2,is_hit3,is_hit4
0,0,今永 昇太,DeNA,20202173,1回表,R,広島,False,False,False,0,0,0,False,False,False,False,False
1,1,今永 昇太,DeNA,20202173,1回表,R,広島,False,False,False,1,0,0,False,False,False,False,False
2,2,今永 昇太,DeNA,20202173,1回表,R,広島,False,False,False,1,0,1,False,False,False,False,False
3,3,今永 昇太,DeNA,20202173,1回表,R,広島,False,False,False,2,0,1,False,False,False,False,False
4,4,今永 昇太,DeNA,20202173,1回表,R,広島,False,False,False,2,0,2,True,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20395,20395,森 唯斗,ソフトバンク,20202118,9回裏,R,日本ハム,False,False,False,1,2,0,False,False,False,False,False
20396,20396,森 唯斗,ソフトバンク,20202118,9回裏,R,日本ハム,False,False,False,1,2,1,False,False,False,False,False
20397,20397,森 唯斗,ソフトバンク,20202118,9回裏,R,日本ハム,False,False,False,2,2,1,False,False,False,False,False
20398,20398,森 唯斗,ソフトバンク,20202118,9回裏,R,日本ハム,False,False,False,2,2,2,False,False,False,False,False


# いろいろな被打率
> チーム被打率 <br>
> ・被打率（単純な被打率）<br>
> ・右（左）バッター被打率 <br>
> ・対戦チーム被打率 <br>
> ・チャンス（出塁状態）被打率 <br>
> ・カウント別被打率 <br>
>
> 以上をヒットの種類（5種類）で作成する

In [7]:
# 打率計算関数
def clc_batting_avg(indt, hit, cols, col):
    
    # 入力データ
    _tmp0 = indt.copy()
    
    # 指定した区分ごとに、ヒットか否かを集約
    _tmp1 = _tmp0.groupby(cols)[hit].max().to_frame().reset_index()
    
    # ピッチャー単位に打率を計算
    return _tmp1.groupby(col)[hit].mean().to_frame()

# ヒットの種類を定義

In [8]:
cols_hit = ['is_hit'+str(i) for i in range(5)]
cols_hit

['is_hit0', 'is_hit1', 'is_hit2', 'is_hit3', 'is_hit4']

# 　

# 0．チーム被打率
> ・pitcher, gameID, inning ⇒　一打席

In [9]:
# ピッチャーチームごとに
_pitcher0 = pd.concat(
    
                [
    
                    clc_batting_avg(

                        indt=train_df,

                        hit=_col,

                        cols=['pitcherTeam', 'pitcher', 'gameID', 'inning'],

                        col=['pitcherTeam']

                    )

                    # ヒットの種類別
                    for _col in cols_hit

                ]
                # 横結合
                , axis=1
    
            ).reset_index()

_pitcher0

,pitcherTeam,is_hit0,is_hit1,is_hit2,is_hit3,is_hit4
0,DeNA,0.600000,0.421053,0.157895,0.042105,0.094737
1,オリックス,0.494845,0.381443,0.113402,0.000000,0.061856
2,ソフトバンク,0.542553,0.414894,0.138298,0.010638,0.106383
3,ヤクルト,0.573034,0.483146,0.191011,0.000000,0.101124
4,ロッテ,0.569892,0.451613,0.129032,0.010753,0.118280
5,中日,0.620690,0.517241,0.149425,0.000000,0.160920
6,巨人,0.505051,0.404040,0.121212,0.000000,0.111111
7,広島,0.536585,0.451220,0.121951,0.024390,0.073171
8,日本ハム,0.617978,0.505618,0.179775,0.000000,0.067416
9,楽天,0.521739,0.347826,0.152174,0.010870,0.097826


### 全体データに結合

In [10]:
_FTR0 = _all_df[['id', 'pitcher', 'pitcherTeam']].merge(

            _pitcher0.rename(columns={'is_hit'+str(i) : 'FTR_pt_btingavg0'+str(i) for i in range(5)}),
            
            how='left',
            on='pitcherTeam'
        )

_FTR0

,id,pitcher,pitcherTeam,FTR_pt_btingavg00,FTR_pt_btingavg01,FTR_pt_btingavg02,FTR_pt_btingavg03,FTR_pt_btingavg04
0,0,今永 昇太,DeNA,0.600000,0.421053,0.157895,0.042105,0.094737
1,1,今永 昇太,DeNA,0.600000,0.421053,0.157895,0.042105,0.094737
2,2,今永 昇太,DeNA,0.600000,0.421053,0.157895,0.042105,0.094737
3,3,今永 昇太,DeNA,0.600000,0.421053,0.157895,0.042105,0.094737
4,4,今永 昇太,DeNA,0.600000,0.421053,0.157895,0.042105,0.094737
...,...,...,...,...,...,...,...,...
54203,33803,床田 寛樹,広島,0.536585,0.451220,0.121951,0.024390,0.073171
54204,33804,堀岡 隼人,巨人,0.505051,0.404040,0.121212,0.000000,0.111111
54205,33805,ディプラン,巨人,0.505051,0.404040,0.121212,0.000000,0.111111
54206,33806,田村 伊知郎,西武,0.505155,0.360825,0.175258,0.010309,0.134021


In [11]:
_FTR0[['FTR_pt_btingavg0'+str(i) for i in range(5)]].isnull().sum()

FTR_pt_btingavg00    0
FTR_pt_btingavg01    0
FTR_pt_btingavg02    0
FTR_pt_btingavg03    0
FTR_pt_btingavg04    0
dtype: int64

☝：12球団すべて結合済み

# 　

## 1．単純な被打率
> ・gameID, inning　⇒　一打席

In [12]:
# ピッチーごとに、一打席に集約
_pitcher1 = pd.concat(

                [
                    clc_batting_avg(

                        indt=train_df,
                        
                        hit=_col,

                        cols=['pitcher', 'gameID', 'inning'], 

                        col=['pitcher']

                    )

                    for _col in cols_hit

                ], axis=1
    
            ).reset_index()

_pitcher1.head(100)

,pitcher,is_hit0,is_hit1,is_hit2,is_hit3,is_hit4
0,アルバース,0.545455,0.363636,0.000000,0.000000,0.181818
1,イノーア,0.833333,0.666667,0.166667,0.000000,0.166667
2,エスコバー,1.000000,0.333333,0.333333,0.000000,0.333333
3,エドワーズ,1.000000,1.000000,0.000000,0.000000,0.000000
4,ガルシア,0.500000,0.500000,0.250000,0.000000,0.250000
5,ガンケル,0.750000,0.750000,0.000000,0.000000,0.000000
6,ギャレット,0.166667,0.166667,0.000000,0.000000,0.000000
7,ゴンサレス,1.000000,1.000000,0.500000,0.000000,0.000000
8,サンチェス,0.500000,0.416667,0.000000,0.000000,0.083333
9,シャギワ,0.250000,0.250000,0.000000,0.000000,0.000000


### 全体データに結合

In [13]:
_FTR1 = _all_df[['id', 'pitcher']].merge(

            _pitcher1.rename(columns={'is_hit'+str(i) : 'FTR_pt_btingavg1'+str(i) for i in range(5)}),
            
            how='left',
            on='pitcher'
        )

_FTR1

,id,pitcher,FTR_pt_btingavg10,FTR_pt_btingavg11,FTR_pt_btingavg12,FTR_pt_btingavg13,FTR_pt_btingavg14
0,0,今永 昇太,0.538462,0.461538,0.076923,0.153846,0.000000
1,1,今永 昇太,0.538462,0.461538,0.076923,0.153846,0.000000
2,2,今永 昇太,0.538462,0.461538,0.076923,0.153846,0.000000
3,3,今永 昇太,0.538462,0.461538,0.076923,0.153846,0.000000
4,4,今永 昇太,0.538462,0.461538,0.076923,0.153846,0.000000
...,...,...,...,...,...,...,...
54203,33803,床田 寛樹,0.714286,0.714286,0.142857,0.000000,0.142857
54204,33804,堀岡 隼人,NaN,NaN,NaN,NaN,NaN
54205,33805,ディプラン,NaN,NaN,NaN,NaN,NaN
54206,33806,田村 伊知郎,1.000000,1.000000,0.500000,0.000000,0.000000


In [14]:
_FTR1[['FTR_pt_btingavg1'+str(i) for i in range(5)]].isnull().sum()

FTR_pt_btingavg10    9288
FTR_pt_btingavg11    9288
FTR_pt_btingavg12    9288
FTR_pt_btingavg13    9288
FTR_pt_btingavg14    9288
dtype: int64

☝：訓練データにいないピッチャー（チーム被打率で補完）

# 　

## 2．バッターの利き手別被打率
> ・gameID, inning, batterHand ⇒　バッターの利き手別：一打席

In [15]:
_pitcher2 = pd.concat(
    
                [
                    clc_batting_avg(

                        # 入力データ：必要データのみ訓練データ
                        indt=train_df, 
                        
                        hit=_col,

                        # 集約単位：打席×バッターの利き手
                        cols=['pitcher', 'gameID', 'inning', 'batterHand'], 

                        # 出力集約状態：投手×バッターの利き手
                        col=['pitcher', 'batterHand']

                    )
                    
                    for _col in cols_hit
                    
                ], axis=1
    
            ).reset_index()

_pitcher2.head(100)

,pitcher,batterHand,is_hit0,is_hit1,is_hit2,is_hit3,is_hit4
0,アルバース,L,0.100000,0.100000,0.000000,0.000000,0.000000
1,アルバース,R,0.600000,0.400000,0.000000,0.000000,0.200000
2,イノーア,L,0.666667,0.500000,0.166667,0.000000,0.000000
3,イノーア,R,0.666667,0.500000,0.000000,0.000000,0.166667
4,エスコバー,L,0.666667,0.333333,0.000000,0.000000,0.333333
5,エスコバー,R,1.000000,0.000000,0.500000,0.000000,0.500000
6,エドワーズ,L,1.000000,1.000000,0.000000,0.000000,0.000000
7,エドワーズ,R,0.000000,0.000000,0.000000,0.000000,0.000000
8,ガルシア,L,0.250000,0.250000,0.000000,0.000000,0.000000
9,ガルシア,R,0.500000,0.500000,0.250000,0.000000,0.250000


☝：対戦するバッターの利き手ごとに結合するので、ピッチャー単位には集約しない

### 全体データに結合

In [16]:
_FTR2 = _all_df[['id', 'pitcher', 'batterHand']].merge(

            _pitcher2.rename(columns={'is_hit'+str(i) : 'FTR_pt_btingavg2' + str(i) for i in range(5)}),
            
            how='left',
    
            on=['pitcher', 'batterHand']
    
        )

_FTR2

,id,pitcher,batterHand,FTR_pt_btingavg20,FTR_pt_btingavg21,FTR_pt_btingavg22,FTR_pt_btingavg23,FTR_pt_btingavg24
0,0,今永 昇太,R,0.538462,0.461538,0.0,0.076923,0.0
1,1,今永 昇太,R,0.538462,0.461538,0.0,0.076923,0.0
2,2,今永 昇太,R,0.538462,0.461538,0.0,0.076923,0.0
3,3,今永 昇太,R,0.538462,0.461538,0.0,0.076923,0.0
4,4,今永 昇太,R,0.538462,0.461538,0.0,0.076923,0.0
...,...,...,...,...,...,...,...,...
54203,33803,床田 寛樹,L,0.666667,0.666667,0.0,0.000000,0.0
54204,33804,堀岡 隼人,R,NaN,NaN,NaN,NaN,NaN
54205,33805,ディプラン,R,NaN,NaN,NaN,NaN,NaN
54206,33806,田村 伊知郎,L,1.000000,0.500000,0.5,0.000000,0.0


In [17]:
_FTR2[['FTR_pt_btingavg2'+str(i) for i in range(5)]].isnull().sum()

FTR_pt_btingavg20    9418
FTR_pt_btingavg21    9418
FTR_pt_btingavg22    9418
FTR_pt_btingavg23    9418
FTR_pt_btingavg24    9418
dtype: int64

☝：対戦したことの無い利き手（投手被打率で補完）

# 　

## 3．対戦チーム被打率
> ・gameID, inning, batterTeam　⇒　バッターのチーム別：一打席

In [18]:
_pitcher3 = pd.concat(
    
                [
                    clc_batting_avg(

                        indt = train_df,

                        hit=_col,

                        cols=['pitcher', 'gameID', 'inning', 'batterTeam'],

                        col=['pitcher', 'batterTeam']

                    )
                    
                    for _col in cols_hit
                    
                ], axis=1
            
            ).reset_index()

_pitcher3.head(100)

,pitcher,batterTeam,is_hit0,is_hit1,is_hit2,is_hit3,is_hit4
0,アルバース,ロッテ,0.666667,0.333333,0.000000,0.000000,0.333333
1,アルバース,西武,0.400000,0.400000,0.000000,0.000000,0.000000
2,イノーア,阪神,0.833333,0.666667,0.166667,0.000000,0.166667
3,エスコバー,巨人,1.000000,0.000000,0.000000,0.000000,1.000000
4,エスコバー,阪神,1.000000,0.500000,0.500000,0.000000,0.000000
5,エドワーズ,巨人,1.000000,1.000000,0.000000,0.000000,0.000000
6,ガルシア,巨人,0.500000,0.500000,0.250000,0.000000,0.250000
7,ガンケル,ヤクルト,0.750000,0.750000,0.000000,0.000000,0.000000
8,ギャレット,オリックス,0.000000,0.000000,0.000000,0.000000,0.000000
9,ギャレット,ソフトバンク,0.250000,0.250000,0.000000,0.000000,0.000000


☝：対戦チームごとに結合するので、ピッチャー単位には集約しない

### 全体データに結合

In [19]:
_FTR3 = _all_df[['id', 'pitcher', 'batterTeam']].merge(

            _pitcher3.rename(columns={'is_hit'+str(i) : 'FTR_pt_btingavg3'+str(i) for i in range(5)}),
            
            how='left',
    
            on=['pitcher', 'batterTeam']
    
        )

_FTR3

,id,pitcher,batterTeam,FTR_pt_btingavg30,FTR_pt_btingavg31,FTR_pt_btingavg32,FTR_pt_btingavg33,FTR_pt_btingavg34
0,0,今永 昇太,広島,0.8,0.6,0.2,0.4,0.0
1,1,今永 昇太,広島,0.8,0.6,0.2,0.4,0.0
2,2,今永 昇太,広島,0.8,0.6,0.2,0.4,0.0
3,3,今永 昇太,広島,0.8,0.6,0.2,0.4,0.0
4,4,今永 昇太,広島,0.8,0.6,0.2,0.4,0.0
...,...,...,...,...,...,...,...,...
54203,33803,床田 寛樹,ヤクルト,NaN,NaN,NaN,NaN,NaN
54204,33804,堀岡 隼人,広島,NaN,NaN,NaN,NaN,NaN
54205,33805,ディプラン,広島,NaN,NaN,NaN,NaN,NaN
54206,33806,田村 伊知郎,ソフトバンク,NaN,NaN,NaN,NaN,NaN


In [20]:
_FTR3[['FTR_pt_btingavg3'+str(i) for i in range(5)]].isnull().sum()

FTR_pt_btingavg30    25584
FTR_pt_btingavg31    25584
FTR_pt_btingavg32    25584
FTR_pt_btingavg33    25584
FTR_pt_btingavg34    25584
dtype: int64

☝：訓練データで対戦したことのないチーム（バッター被打率で補完）

# 　

## 4．出塁状態ごとの被打率
> ・gameID, inning, b1, b2, b3　⇒　出塁状態別：一打席

In [21]:
_pitcher4 = pd.concat(
    
                [
                    clc_batting_avg(

                        indt = train_df,
                        
                        hit = _col,

                        cols = ['pitcher', 'gameID', 'inning', 'b1', 'b2', 'b3'],

                        col = ['pitcher', 'b1', 'b2', 'b3']

                    )
                    
                    for _col in cols_hit
                    
                ], axis=1
    
            ).reset_index()

_pitcher4.head(100)

,pitcher,b1,b2,b3,is_hit0,is_hit1,is_hit2,is_hit3,is_hit4
0,アルバース,False,False,False,0.454545,0.272727,0.000000,0.0,0.181818
1,アルバース,True,False,False,0.333333,0.166667,0.000000,0.0,0.166667
2,アルバース,True,True,False,1.000000,1.000000,0.000000,0.0,0.000000
3,アルバース,True,True,True,1.000000,1.000000,0.000000,0.0,0.000000
4,イノーア,False,False,False,0.833333,0.666667,0.166667,0.0,0.000000
5,イノーア,False,False,True,1.000000,0.000000,1.000000,0.0,1.000000
6,イノーア,False,True,False,0.000000,0.000000,0.000000,0.0,0.000000
7,イノーア,True,False,False,0.250000,0.250000,0.000000,0.0,0.000000
8,エスコバー,False,False,False,1.000000,0.333333,0.333333,0.0,0.333333
9,エスコバー,False,True,False,0.000000,0.000000,0.000000,0.0,0.000000


### 全体データに結合

In [22]:
_FTR4 = _all_df[['id', 'pitcher', 'b1', 'b2', 'b3']].merge(

            _pitcher4.rename(columns={'is_hit'+str(i) : 'FTR_pt_btingavg4'+str(i) for i in range(5)}),
            
            how='left',
    
            on=['pitcher', 'b1', 'b2', 'b3']
    
        )

_FTR4

,id,pitcher,b1,b2,b3,FTR_pt_btingavg40,FTR_pt_btingavg41,FTR_pt_btingavg42,FTR_pt_btingavg43,FTR_pt_btingavg44
0,0,今永 昇太,False,False,False,0.538462,0.384615,0.000000,0.153846,0.0
1,1,今永 昇太,False,False,False,0.538462,0.384615,0.000000,0.153846,0.0
2,2,今永 昇太,False,False,False,0.538462,0.384615,0.000000,0.153846,0.0
3,3,今永 昇太,False,False,False,0.538462,0.384615,0.000000,0.153846,0.0
4,4,今永 昇太,False,False,False,0.538462,0.384615,0.000000,0.153846,0.0
...,...,...,...,...,...,...,...,...,...,...
54203,33803,床田 寛樹,False,False,False,0.714286,0.571429,0.142857,0.000000,0.0
54204,33804,堀岡 隼人,False,False,False,NaN,NaN,NaN,NaN,NaN
54205,33805,ディプラン,True,False,False,NaN,NaN,NaN,NaN,NaN
54206,33806,田村 伊知郎,False,True,False,1.000000,0.000000,1.000000,0.000000,0.0


In [23]:
_FTR4[['FTR_pt_btingavg4'+str(i) for i in range(5)]].isnull().sum()

FTR_pt_btingavg40    11191
FTR_pt_btingavg41    11191
FTR_pt_btingavg42    11191
FTR_pt_btingavg43    11191
FTR_pt_btingavg44    11191
dtype: int64

☝：訓練データで発生していない出塁状態（バッター被打率で補完）

# 　

# 5．カウント別被打率
> ・gameID, inning, b1, b2, b3　⇒　出塁状態別：一打席

In [24]:
_pitcher5 = pd.concat(
    
                [
                    clc_batting_avg(

                        indt = train_df,
                        
                        hit = _col,

                        cols = ['pitcher', 'gameID', 'inning', 'B', 'O', 'S'],

                        col = ['pitcher', 'B', 'O', 'S']

                    )
                    
                    for _col in cols_hit
                    
                ], axis=1
    
            ).reset_index()

_pitcher5.head(100)

,pitcher,B,O,S,is_hit0,is_hit1,is_hit2,is_hit3,is_hit4
0,アルバース,0,0,0,0.000000,0.000000,0.000000,0.0,0.000000
1,アルバース,0,0,1,0.000000,0.000000,0.000000,0.0,0.000000
2,アルバース,0,0,2,0.000000,0.000000,0.000000,0.0,0.000000
3,アルバース,0,1,0,0.000000,0.000000,0.000000,0.0,0.000000
4,アルバース,0,1,1,0.142857,0.142857,0.000000,0.0,0.000000
5,アルバース,0,1,2,0.000000,0.000000,0.000000,0.0,0.000000
6,アルバース,0,2,0,0.100000,0.100000,0.000000,0.0,0.000000
7,アルバース,0,2,1,0.142857,0.142857,0.000000,0.0,0.000000
8,アルバース,0,2,2,0.000000,0.000000,0.000000,0.0,0.000000
9,アルバース,1,0,0,0.000000,0.000000,0.000000,0.0,0.000000


## 全体データに結合

In [25]:
_FTR5 = _all_df[['id', 'pitcher', 'B', 'O', 'S']].merge(

            _pitcher5.rename(columns={'is_hit'+str(i) : 'FTR_pt_btingavg5'+str(i) for i in range(5)}),
            
            how='left',
    
            on=['pitcher', 'B', 'O', 'S']
    
        )

_FTR5

,id,pitcher,B,O,S,FTR_pt_btingavg50,FTR_pt_btingavg51,FTR_pt_btingavg52,FTR_pt_btingavg53,FTR_pt_btingavg54
0,0,今永 昇太,0,0,0,0.000000,0.000000,0.0,0.0,0.0
1,1,今永 昇太,1,0,0,0.000000,0.000000,0.0,0.0,0.0
2,2,今永 昇太,1,0,1,0.111111,0.111111,0.0,0.0,0.0
3,3,今永 昇太,2,0,1,0.000000,0.000000,0.0,0.0,0.0
4,4,今永 昇太,2,0,2,0.500000,0.500000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
54203,33803,床田 寛樹,0,0,1,0.000000,0.000000,0.0,0.0,0.0
54204,33804,堀岡 隼人,0,0,0,NaN,NaN,NaN,NaN,NaN
54205,33805,ディプラン,0,0,0,NaN,NaN,NaN,NaN,NaN
54206,33806,田村 伊知郎,3,1,1,NaN,NaN,NaN,NaN,NaN


In [26]:
_FTR4[['FTR_pt_btingavg4'+str(i) for i in range(5)]].isnull().sum()

FTR_pt_btingavg40    11191
FTR_pt_btingavg41    11191
FTR_pt_btingavg42    11191
FTR_pt_btingavg43    11191
FTR_pt_btingavg44    11191
dtype: int64

☝：訓練データで遭遇していないカウント

# 　

# データ結合
> ・作成した状況別打率を、全体データ（_all_df）に結合する

In [27]:
_FTR0.head()

,id,pitcher,pitcherTeam,FTR_pt_btingavg00,FTR_pt_btingavg01,FTR_pt_btingavg02,FTR_pt_btingavg03,FTR_pt_btingavg04
0,0,今永 昇太,DeNA,0.6,0.421053,0.157895,0.042105,0.094737
1,1,今永 昇太,DeNA,0.6,0.421053,0.157895,0.042105,0.094737
2,2,今永 昇太,DeNA,0.6,0.421053,0.157895,0.042105,0.094737
3,3,今永 昇太,DeNA,0.6,0.421053,0.157895,0.042105,0.094737
4,4,今永 昇太,DeNA,0.6,0.421053,0.157895,0.042105,0.094737


In [28]:
_FTR1.head()

,id,pitcher,FTR_pt_btingavg10,FTR_pt_btingavg11,FTR_pt_btingavg12,FTR_pt_btingavg13,FTR_pt_btingavg14
0,0,今永 昇太,0.538462,0.461538,0.076923,0.153846,0.0
1,1,今永 昇太,0.538462,0.461538,0.076923,0.153846,0.0
2,2,今永 昇太,0.538462,0.461538,0.076923,0.153846,0.0
3,3,今永 昇太,0.538462,0.461538,0.076923,0.153846,0.0
4,4,今永 昇太,0.538462,0.461538,0.076923,0.153846,0.0


In [29]:
_FTR2.head()

,id,pitcher,batterHand,FTR_pt_btingavg20,FTR_pt_btingavg21,FTR_pt_btingavg22,FTR_pt_btingavg23,FTR_pt_btingavg24
0,0,今永 昇太,R,0.538462,0.461538,0.0,0.076923,0.0
1,1,今永 昇太,R,0.538462,0.461538,0.0,0.076923,0.0
2,2,今永 昇太,R,0.538462,0.461538,0.0,0.076923,0.0
3,3,今永 昇太,R,0.538462,0.461538,0.0,0.076923,0.0
4,4,今永 昇太,R,0.538462,0.461538,0.0,0.076923,0.0


In [30]:
_FTR3.head()

,id,pitcher,batterTeam,FTR_pt_btingavg30,FTR_pt_btingavg31,FTR_pt_btingavg32,FTR_pt_btingavg33,FTR_pt_btingavg34
0,0,今永 昇太,広島,0.8,0.6,0.2,0.4,0.0
1,1,今永 昇太,広島,0.8,0.6,0.2,0.4,0.0
2,2,今永 昇太,広島,0.8,0.6,0.2,0.4,0.0
3,3,今永 昇太,広島,0.8,0.6,0.2,0.4,0.0
4,4,今永 昇太,広島,0.8,0.6,0.2,0.4,0.0


In [31]:
_FTR4.head()

,id,pitcher,b1,b2,b3,FTR_pt_btingavg40,FTR_pt_btingavg41,FTR_pt_btingavg42,FTR_pt_btingavg43,FTR_pt_btingavg44
0,0,今永 昇太,False,False,False,0.538462,0.384615,0.0,0.153846,0.0
1,1,今永 昇太,False,False,False,0.538462,0.384615,0.0,0.153846,0.0
2,2,今永 昇太,False,False,False,0.538462,0.384615,0.0,0.153846,0.0
3,3,今永 昇太,False,False,False,0.538462,0.384615,0.0,0.153846,0.0
4,4,今永 昇太,False,False,False,0.538462,0.384615,0.0,0.153846,0.0


In [32]:
_FTR5.head()

,id,pitcher,B,O,S,FTR_pt_btingavg50,FTR_pt_btingavg51,FTR_pt_btingavg52,FTR_pt_btingavg53,FTR_pt_btingavg54
0,0,今永 昇太,0,0,0,0.000000,0.000000,0.0,0.0,0.0
1,1,今永 昇太,1,0,0,0.000000,0.000000,0.0,0.0,0.0
2,2,今永 昇太,1,0,1,0.111111,0.111111,0.0,0.0,0.0
3,3,今永 昇太,2,0,1,0.000000,0.000000,0.0,0.0,0.0
4,4,今永 昇太,2,0,2,0.500000,0.500000,0.0,0.0,0.0


In [33]:
_FTR_btingavg = pd.concat(
    
                    [
                        _all_df,
                        
                        # 特徴量のみ抽出
                        _FTR0.filter(like='FTR', axis=1), 
                        _FTR1.filter(like='FTR', axis=1),
                        _FTR2.filter(like='FTR', axis=1),
                        _FTR3.filter(like='FTR', axis=1),
                        _FTR4.filter(like='FTR', axis=1),
                        _FTR5.filter(like='FTR', axis=1)
                    ], 
                    axis=1

            )

_FTR_btingavg.head(100)

,id,pitcher,pitcherTeam,gameID,inning,batterHand,batterTeam,b1,b2,b3,B,O,S,is_hit0,is_hit1,is_hit2,is_hit3,is_hit4,flg_train,FTR_pt_btingavg00,FTR_pt_btingavg01,FTR_pt_btingavg02,FTR_pt_btingavg03,FTR_pt_btingavg04,FTR_pt_btingavg10,FTR_pt_btingavg11,FTR_pt_btingavg12,FTR_pt_btingavg13,FTR_pt_btingavg14,FTR_pt_btingavg20,FTR_pt_btingavg21,FTR_pt_btingavg22,FTR_pt_btingavg23,FTR_pt_btingavg24,FTR_pt_btingavg30,FTR_pt_btingavg31,FTR_pt_btingavg32,FTR_pt_btingavg33,FTR_pt_btingavg34,FTR_pt_btingavg40,FTR_pt_btingavg41,FTR_pt_btingavg42,FTR_pt_btingavg43,FTR_pt_btingavg44,FTR_pt_btingavg50,FTR_pt_btingavg51,FTR_pt_btingavg52,FTR_pt_btingavg53,FTR_pt_btingavg54
0,0,今永 昇太,DeNA,20202173,1回表,R,広島,False,False,False,0,0,0,False,False,False,False,False,1,0.600000,0.421053,0.157895,0.042105,0.094737,0.538462,0.461538,0.076923,0.153846,0.000000,0.538462,0.461538,0.000000,0.076923,0.000000,0.800000,0.600000,0.200000,0.4,0.000000,0.538462,0.384615,0.000000,0.153846,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
1,1,今永 昇太,DeNA,20202173,1回表,R,広島,False,False,False,1,0,0,False,False,False,False,False,1,0.600000,0.421053,0.157895,0.042105,0.094737,0.538462,0.461538,0.076923,0.153846,0.000000,0.538462,0.461538,0.000000,0.076923,0.000000,0.800000,0.600000,0.200000,0.4,0.000000,0.538462,0.384615,0.000000,0.153846,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
2,2,今永 昇太,DeNA,20202173,1回表,R,広島,False,False,False,1,0,1,False,False,False,False,False,1,0.600000,0.421053,0.157895,0.042105,0.094737,0.538462,0.461538,0.076923,0.153846,0.000000,0.538462,0.461538,0.000000,0.076923,0.000000,0.800000,0.600000,0.200000,0.4,0.000000,0.538462,0.384615,0.000000,0.153846,0.000000,0.111111,0.111111,0.000000,0.0,0.000000
3,3,今永 昇太,DeNA,20202173,1回表,R,広島,False,False,False,2,0,1,False,False,False,False,False,1,0.600000,0.421053,0.157895,0.042105,0.094737,0.538462,0.461538,0.076923,0.153846,0.000000,0.538462,0.461538,0.000000,0.076923,0.000000,0.800000,0.600000,0.200000,0.4,0.000000,0.538462,0.384615,0.000000,0.153846,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
4,4,今永 昇太,DeNA,20202173,1回表,R,広島,False,False,False,2,0,2,True,True,False,False,False,1,0.600000,0.421053,0.157895,0.042105,0.094737,0.538462,0.461538,0.076923,0.153846,0.000000,0.538462,0.461538,0.000000,0.076923,0.000000,0.800000,0.600000,0.200000,0.4,0.000000,0.538462,0.384615,0.000000,0.153846,0.000000,0.500000,0.500000,0.000000,0.0,0.000000
5,5,今永 昇太,DeNA,20202173,1回表,R,広島,True,False,False,0,0,0,False,False,False,False,False,1,0.600000,0.421053,0.157895,0.042105,0.094737,0.538462,0.461538,0.076923,0.153846,0.000000,0.538462,0.461538,0.000000,0.076923,0.000000,0.800000,0.600000,0.200000,0.4,0.000000,0.166667,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
6,6,今永 昇太,DeNA,20202173,1回表,R,広島,True,False,False,0,0,1,False,False,False,False,False,1,0.600000,0.421053,0.157895,0.042105,0.094737,0.538462,0.461538,0.076923,0.153846,0.000000,0.538462,0.461538,0.000000,0.076923,0.000000,0.800000,0.600000,0.200000,0.4,0.000000,0.166667,0.166667,0.000000,0.000000,0.000000,0.111111,0.111111,0.000000,0.0,0.000000
7,7,今永 昇太,DeNA,20202173,1回表,L,広島,True,False,False,0,1,0,False,False,False,False,False,1,0.600000,0.421053,0.157895,0.042105,0.094737,0.538462,0.461538,0.076923,0.153846,0.000000,0.200000,0.100000,0.100000,0.100000,0.000000,0.800000,0.600000,0.200000,0.4,0.000000,0.166667,0.166667,0.000000,0.000000,0.000000,0.076923,0.076923,0.000000,0.0,0.000000
8,8,今永 昇太,DeNA,20202173,1回表,L,広島,True,False,False,0,1,1,False,False,False,False,False,1,0.600000,0.421053,0.157895,0.042105,0.094737,0.538462,0.461538,0.076923,0.153846,0.000000,0.200000,0.100000,0.100000,0.100000,0.000000,0.800000,0.600000,0.200000,0.4,0.000000,0.166667,0.166667,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.2,0.000000
9,9,今永 昇太,DeNA,20202173,1回表,L,広島,True,False,False,0,1,2,False,False,False,False,False,1,0.600000,0.421053,0.157895,0.042105,0.094737,0.538462,0.461538,0.076923,0.153846,0.000000,0.200000,0.100000,0.10

# 　

# 欠損値処置
> ・投手被打率：チーム被打率 <br>
> ・バッター利き手別被打率：投手被打率（補完済み）<br>
> ・対戦チーム被打率：投手被打率（〃）<br>
> ・出塁状況別被打率：投手被打率（〃）<br>
> ・カウント別被打率：投手被打率（〃）<br>

In [34]:
# 処理後データ作成
FTR_btingavg = _FTR_btingavg.copy()

In [35]:
# 1．投手被打率：チーム被打率
for i in range(5):

    FTR_btingavg['FTR_pt_btingavg1'+str(i)] = FTR_btingavg['FTR_pt_btingavg1'+str(i)].fillna(FTR_btingavg['FTR_pt_btingavg0'+str(i)])

In [36]:
for i in range(5):
    
    # 2．バッター利き手別被打率：投手被打率（補完済み）
    FTR_btingavg['FTR_pt_btingavg2'+str(i)] = FTR_btingavg['FTR_pt_btingavg2'+str(i)].fillna(FTR_btingavg['FTR_pt_btingavg1'+str(i)])
    
    # 3．対戦チーム被打率：投手被打率（〃）
    FTR_btingavg['FTR_pt_btingavg3'+str(i)] = FTR_btingavg['FTR_pt_btingavg3'+str(i)].fillna(FTR_btingavg['FTR_pt_btingavg1'+str(i)])
    
    # 4．出塁状況別被打率：投手被打率（〃）
    FTR_btingavg['FTR_pt_btingavg4'+str(i)] = FTR_btingavg['FTR_pt_btingavg4'+str(i)].fillna(FTR_btingavg['FTR_pt_btingavg1'+str(i)])
    
    # 5．カウント別被打率：投手被打率（〃）
    FTR_btingavg['FTR_pt_btingavg5'+str(i)] = FTR_btingavg['FTR_pt_btingavg5'+str(i)].fillna(FTR_btingavg['FTR_pt_btingavg1'+str(i)])

補完処理の確認

In [37]:
FTR_btingavg[_FTR1['FTR_pt_btingavg10'].isnull()]

,id,pitcher,pitcherTeam,gameID,inning,batterHand,batterTeam,b1,b2,b3,B,O,S,is_hit0,is_hit1,is_hit2,is_hit3,is_hit4,flg_train,FTR_pt_btingavg00,FTR_pt_btingavg01,FTR_pt_btingavg02,FTR_pt_btingavg03,FTR_pt_btingavg04,FTR_pt_btingavg10,FTR_pt_btingavg11,FTR_pt_btingavg12,FTR_pt_btingavg13,FTR_pt_btingavg14,FTR_pt_btingavg20,FTR_pt_btingavg21,FTR_pt_btingavg22,FTR_pt_btingavg23,FTR_pt_btingavg24,FTR_pt_btingavg30,FTR_pt_btingavg31,FTR_pt_btingavg32,FTR_pt_btingavg33,FTR_pt_btingavg34,FTR_pt_btingavg40,FTR_pt_btingavg41,FTR_pt_btingavg42,FTR_pt_btingavg43,FTR_pt_btingavg44,FTR_pt_btingavg50,FTR_pt_btingavg51,FTR_pt_btingavg52,FTR_pt_btingavg53,FTR_pt_btingavg54
20403,3,クック,ヤクルト,20202650,3回裏,L,中日,True,False,False,0,2,0,False,False,False,False,False,0,0.573034,0.483146,0.191011,0.000000,0.101124,0.573034,0.483146,0.191011,0.000000,0.101124,0.573034,0.483146,0.191011,0.000000,0.101124,0.573034,0.483146,0.191011,0.000000,0.101124,0.573034,0.483146,0.191011,0.000000,0.101124,0.573034,0.483146,0.191011,0.000000,0.101124
20407,7,寺岡 寛治,楽天,20202680,6回表,L,ソフトバンク,False,False,False,1,2,1,False,False,False,False,False,0,0.521739,0.347826,0.152174,0.010870,0.097826,0.521739,0.347826,0.152174,0.010870,0.097826,0.521739,0.347826,0.152174,0.010870,0.097826,0.521739,0.347826,0.152174,0.010870,0.097826,0.521739,0.347826,0.152174,0.010870,0.097826,0.521739,0.347826,0.152174,0.010870,0.097826
20409,9,齋藤 綱記,オリックス,20202704,3回表,R,楽天,True,False,False,1,1,2,False,False,False,False,False,0,0.494845,0.381443,0.113402,0.000000,0.061856,0.494845,0.381443,0.113402,0.000000,0.061856,0.494845,0.381443,0.113402,0.000000,0.061856,0.494845,0.381443,0.113402,0.000000,0.061856,0.494845,0.381443,0.113402,0.000000,0.061856,0.494845,0.381443,0.113402,0.000000,0.061856
20414,14,笠谷 俊介,ソフトバンク,20202335,2回表,R,日本ハム,False,False,False,0,0,1,False,False,False,False,False,0,0.542553,0.414894,0.138298,0.010638,0.106383,0.542553,0.414894,0.138298,0.010638,0.106383,0.542553,0.414894,0.138298,0.010638,0.106383,0.542553,0.414894,0.138298,0.010638,0.106383,0.542553,0.414894,0.138298,0.010638,0.106383,0.542553,0.414894,0.138298,0.010638,0.106383
20415,15,千賀 滉大,ソフトバンク,20202716,2回裏,R,西武,False,True,False,0,2,2,False,False,False,False,False,0,0.542553,0.414894,0.138298,0.010638,0.106383,0.542553,0.414894,0.138298,0.010638,0.106383,0.542553,0.414894,0.138298,0.010638,0.106383,0.542553,0.414894,0.138298,0.010638,0.106383,0.542553,0.414894,0.138298,0.010638,0.106383,0.542553,0.414894,0.138298,0.010638,0.106383
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54182,33782,清水 達也,中日,20202779,2回裏,R,広島,False,False,False,0,1,0,False,False,False,False,False,0,0.620690,0.517241,0.149425,0.000000,0.160920,0.620690,0.517241,0.149425,0.000000,0.160920,0.620690,0.517241,0.149425,0.000000,0.160920,0.620690,0.517241,0.149425,0.000000,0.160920,0.620690,0.517241,0.149425,0.000000,0.160920,0.620690,0.517241,0.149425,0.000000,0.160920
54194,33794,漆原 大晟,オリックス,20202513,6回裏,L,ソフトバンク,False,False,False,1,0,2,False,False,False,False,False,0,0.494845,0.381443,0.113402,0.000000,0.061856,0.494845,0.381443,0.113402,0.000000,0.061856,0.494845,0.381443,0.113402,0.000000,0.061856,0.494845,0.381443,0.113402,0.000000,0.061856,0.494845,0.381443,0.113402,0.000000,0.061856,0.494845,0.381443,0.113402,0.000000,0.061856
54202,33802,上原 健太,日本ハム,20202656,6回裏,R,西武,False,False,False,0,0,1,False,False,False,False,False,0,0.617978,0.505618,0.179775,0.000000,0.067416,0.617978,0.505618,0.179775,0.000000,0.067416,0.617978,0.505618,0.179775,0.000000,0.067416,0.617978,0.505618,0.179775,0.000000,0.067416,0.617978,0.505618,0.179775,0.000000,0.067416,0.617978,0.505618,0.179775,0.000000,0.067416
54204,33804,堀岡 隼人,巨人,20202640,9回表,R,広島,False,False,False,0,0,0,False,False,False,False,False,0,0.505051,0.404040,0.121212,0.000000,0.111111,0.505051,0.404040,0.121212,0.000000,0.111111,0.505051,0.404040,0.12121

In [38]:
FTR_btingavg[_FTR2['FTR_pt_btingavg20'].isnull()]

,id,pitcher,pitcherTeam,gameID,inning,batterHand,batterTeam,b1,b2,b3,B,O,S,is_hit0,is_hit1,is_hit2,is_hit3,is_hit4,flg_train,FTR_pt_btingavg00,FTR_pt_btingavg01,FTR_pt_btingavg02,FTR_pt_btingavg03,FTR_pt_btingavg04,FTR_pt_btingavg10,FTR_pt_btingavg11,FTR_pt_btingavg12,FTR_pt_btingavg13,FTR_pt_btingavg14,FTR_pt_btingavg20,FTR_pt_btingavg21,FTR_pt_btingavg22,FTR_pt_btingavg23,FTR_pt_btingavg24,FTR_pt_btingavg30,FTR_pt_btingavg31,FTR_pt_btingavg32,FTR_pt_btingavg33,FTR_pt_btingavg34,FTR_pt_btingavg40,FTR_pt_btingavg41,FTR_pt_btingavg42,FTR_pt_btingavg43,FTR_pt_btingavg44,FTR_pt_btingavg50,FTR_pt_btingavg51,FTR_pt_btingavg52,FTR_pt_btingavg53,FTR_pt_btingavg54
986,986,東浜 巨,ソフトバンク,20202170,3回表,NaN,ロッテ,True,False,True,0,2,0,False,False,False,False,False,1,0.542553,0.414894,0.138298,0.010638,0.106383,0.545455,0.272727,0.181818,0.0,0.181818,0.545455,0.272727,0.181818,0.0,0.181818,0.400000,0.200000,0.200000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
987,987,東浜 巨,ソフトバンク,20202170,3回表,NaN,ロッテ,True,False,True,0,2,1,False,False,False,False,False,1,0.542553,0.414894,0.138298,0.010638,0.106383,0.545455,0.272727,0.181818,0.0,0.181818,0.545455,0.272727,0.181818,0.0,0.181818,0.400000,0.200000,0.200000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.200000,0.000000,0.000000,0.0,0.200000
988,988,東浜 巨,ソフトバンク,20202170,3回表,NaN,ロッテ,True,False,True,1,2,1,False,False,False,False,False,1,0.542553,0.414894,0.138298,0.010638,0.106383,0.545455,0.272727,0.181818,0.0,0.181818,0.545455,0.272727,0.181818,0.0,0.181818,0.400000,0.200000,0.200000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
2975,2975,松井 裕樹,楽天,20202165,1回裏,NaN,オリックス,True,False,True,0,2,0,False,False,False,False,False,1,0.521739,0.347826,0.152174,0.010870,0.097826,0.666667,0.555556,0.000000,0.0,0.222222,0.666667,0.555556,0.000000,0.0,0.222222,0.750000,0.750000,0.000000,0.0,0.000000,1.000000,1.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
2976,2976,松井 裕樹,楽天,20202165,1回裏,NaN,オリックス,True,False,True,1,2,0,False,False,False,False,False,1,0.521739,0.347826,0.152174,0.010870,0.097826,0.666667,0.555556,0.000000,0.0,0.222222,0.666667,0.555556,0.000000,0.0,0.222222,0.750000,0.750000,0.000000,0.0,0.000000,1.000000,1.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54182,33782,清水 達也,中日,20202779,2回裏,R,広島,False,False,False,0,1,0,False,False,False,False,False,0,0.620690,0.517241,0.149425,0.000000,0.160920,0.620690,0.517241,0.149425,0.0,0.160920,0.620690,0.517241,0.149425,0.0,0.160920,0.620690,0.517241,0.149425,0.0,0.160920,0.620690,0.517241,0.149425,0.0,0.160920,0.620690,0.517241,0.149425,0.0,0.160920
54194,33794,漆原 大晟,オリックス,20202513,6回裏,L,ソフトバンク,False,False,False,1,0,2,False,False,False,False,False,0,0.494845,0.381443,0.113402,0.000000,0.061856,0.494845,0.381443,0.113402,0.0,0.061856,0.494845,0.381443,0.113402,0.0,0.061856,0.494845,0.381443,0.113402,0.0,0.061856,0.494845,0.381443,0.113402,0.0,0.061856,0.494845,0.381443,0.113402,0.0,0.061856
54202,33802,上原 健太,日本ハム,20202656,6回裏,R,西武,False,False,False,0,0,1,False,False,False,False,False,0,0.617978,0.505618,0.179775,0.000000,0.067416,0.617978,0.505618,0.179775,0.0,0.067416,0.617978,0.505618,0.179775,0.0,0.067416,0.617978,0.505618,0.179775,0.0,0.067416,0.617978,0.505618,0.179775,0.0,0.067416,0.617978,0.505618,0.179775,0.0,0.067416
54204,33804,堀岡 隼人,巨人,20202640,9回表,R,広島,False,False,False,0,0,0,False,False,False,False,False,0,0.505051,0.404040,0.121212,0.000000,0.111111,0.505051,0.404040,0.121212,0.0,0.111111,0.505051,0.404040,0.121212,0.0,0.111111,0.505051,0.404040,0.121212,0.0,0.111111,0.505051,0.404040,0.121212,0.0,0.111111,0.505051,0.404040,0.121212,0.0,0.111111


In [39]:
FTR_btingavg[_FTR3['FTR_pt_btingavg30'].isnull()]

,id,pitcher,pitcherTeam,gameID,inning,batterHand,batterTeam,b1,b2,b3,B,O,S,is_hit0,is_hit1,is_hit2,is_hit3,is_hit4,flg_train,FTR_pt_btingavg00,FTR_pt_btingavg01,FTR_pt_btingavg02,FTR_pt_btingavg03,FTR_pt_btingavg04,FTR_pt_btingavg10,FTR_pt_btingavg11,FTR_pt_btingavg12,FTR_pt_btingavg13,FTR_pt_btingavg14,FTR_pt_btingavg20,FTR_pt_btingavg21,FTR_pt_btingavg22,FTR_pt_btingavg23,FTR_pt_btingavg24,FTR_pt_btingavg30,FTR_pt_btingavg31,FTR_pt_btingavg32,FTR_pt_btingavg33,FTR_pt_btingavg34,FTR_pt_btingavg40,FTR_pt_btingavg41,FTR_pt_btingavg42,FTR_pt_btingavg43,FTR_pt_btingavg44,FTR_pt_btingavg50,FTR_pt_btingavg51,FTR_pt_btingavg52,FTR_pt_btingavg53,FTR_pt_btingavg54
20400,0,遠藤 淳志,広島,20202564,2回表,L,DeNA,False,False,False,1,0,0,False,False,False,False,False,0,0.536585,0.451220,0.121951,0.024390,0.073171,0.800000,0.600000,0.400000,0.0,0.200000,0.600000,0.600000,0.000000,0.0,0.000000,0.800000,0.600000,0.400000,0.0,0.200000,0.600000,0.200000,0.400000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
20401,1,バンデンハーク,ソフトバンク,20202106,3回裏,L,日本ハム,False,False,False,0,0,0,False,False,False,False,False,0,0.542553,0.414894,0.138298,0.010638,0.106383,0.125000,0.125000,0.125000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.125000,0.125000,0.125000,0.0,0.000000,0.125000,0.125000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
20402,2,スアレス,阪神,20203305,9回裏,R,広島,True,False,False,3,2,2,False,False,False,False,False,0,0.573034,0.483146,0.134831,0.011236,0.101124,0.333333,0.333333,0.000000,0.0,0.000000,0.125000,0.125000,0.000000,0.0,0.000000,0.333333,0.333333,0.000000,0.0,0.000000,0.500000,0.500000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
20403,3,クック,ヤクルト,20202650,3回裏,L,中日,True,False,False,0,2,0,False,False,False,False,False,0,0.573034,0.483146,0.191011,0.000000,0.101124,0.573034,0.483146,0.191011,0.0,0.101124,0.573034,0.483146,0.191011,0.0,0.101124,0.573034,0.483146,0.191011,0.0,0.101124,0.573034,0.483146,0.191011,0.0,0.101124,0.573034,0.483146,0.191011,0.0,0.101124
20405,5,小島 和哉,ロッテ,20202505,4回裏,R,楽天,False,False,False,0,2,0,False,False,False,False,False,0,0.569892,0.451613,0.129032,0.010753,0.118280,0.400000,0.400000,0.000000,0.0,0.200000,0.400000,0.400000,0.000000,0.0,0.000000,0.400000,0.400000,0.000000,0.0,0.200000,0.200000,0.200000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54203,33803,床田 寛樹,広島,20202023,5回表,L,ヤクルト,False,False,False,0,0,1,False,False,False,False,False,0,0.536585,0.451220,0.121951,0.024390,0.073171,0.714286,0.714286,0.142857,0.0,0.142857,0.666667,0.666667,0.000000,0.0,0.000000,0.714286,0.714286,0.142857,0.0,0.142857,0.714286,0.571429,0.142857,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
54204,33804,堀岡 隼人,巨人,20202640,9回表,R,広島,False,False,False,0,0,0,False,False,False,False,False,0,0.505051,0.404040,0.121212,0.000000,0.111111,0.505051,0.404040,0.121212,0.0,0.111111,0.505051,0.404040,0.121212,0.0,0.111111,0.505051,0.404040,0.121212,0.0,0.111111,0.505051,0.404040,0.121212,0.0,0.111111,0.505051,0.404040,0.121212,0.0,0.111111
54205,33805,ディプラン,巨人,20202864,7回裏,R,広島,True,False,False,0,0,0,False,False,False,False,False,0,0.505051,0.404040,0.121212,0.000000,0.111111,0.505051,0.404040,0.121212,0.0,0.111111,0.505051,0.404040,0.121212,0.0,0.111111,0.505051,0.404040,0.121212,0.0,0.111111,0.505051,0.404040,0.121212,0.0,0.111111,0.505051,0.404040,0.121212,0.0,0.111111
54206,33806,田村 伊知郎,西武,20202806,8回裏,L,ソフトバンク,False,True,False,3,1,1,False,False,False,False,False,0,0.505155,0.360825,0.175258,0.010309,0.134021,1.000000,1.000000,0.500000,0.0,0.000000,1.000000,0.500000,0.500000,0.0,0.000000,1.000000,1.000000,0.500000,0.0,0.000000,1.000000,0.000000,1.000000,0.0,0.000000,1.000000,1.000000,0.500000,0.0,0.000000


In [40]:
FTR_btingavg[_FTR4['FTR_pt_btingavg40'].isnull()]

,id,pitcher,pitcherTeam,gameID,inning,batterHand,batterTeam,b1,b2,b3,B,O,S,is_hit0,is_hit1,is_hit2,is_hit3,is_hit4,flg_train,FTR_pt_btingavg00,FTR_pt_btingavg01,FTR_pt_btingavg02,FTR_pt_btingavg03,FTR_pt_btingavg04,FTR_pt_btingavg10,FTR_pt_btingavg11,FTR_pt_btingavg12,FTR_pt_btingavg13,FTR_pt_btingavg14,FTR_pt_btingavg20,FTR_pt_btingavg21,FTR_pt_btingavg22,FTR_pt_btingavg23,FTR_pt_btingavg24,FTR_pt_btingavg30,FTR_pt_btingavg31,FTR_pt_btingavg32,FTR_pt_btingavg33,FTR_pt_btingavg34,FTR_pt_btingavg40,FTR_pt_btingavg41,FTR_pt_btingavg42,FTR_pt_btingavg43,FTR_pt_btingavg44,FTR_pt_btingavg50,FTR_pt_btingavg51,FTR_pt_btingavg52,FTR_pt_btingavg53,FTR_pt_btingavg54
20403,3,クック,ヤクルト,20202650,3回裏,L,中日,True,False,False,0,2,0,False,False,False,False,False,0,0.573034,0.483146,0.191011,0.000000,0.101124,0.573034,0.483146,0.191011,0.000000,0.101124,0.573034,0.483146,0.191011,0.000000,0.101124,0.573034,0.483146,0.191011,0.000000,0.101124,0.573034,0.483146,0.191011,0.000000,0.101124,0.573034,0.483146,0.191011,0.000000,0.101124
20407,7,寺岡 寛治,楽天,20202680,6回表,L,ソフトバンク,False,False,False,1,2,1,False,False,False,False,False,0,0.521739,0.347826,0.152174,0.010870,0.097826,0.521739,0.347826,0.152174,0.010870,0.097826,0.521739,0.347826,0.152174,0.010870,0.097826,0.521739,0.347826,0.152174,0.010870,0.097826,0.521739,0.347826,0.152174,0.010870,0.097826,0.521739,0.347826,0.152174,0.010870,0.097826
20409,9,齋藤 綱記,オリックス,20202704,3回表,R,楽天,True,False,False,1,1,2,False,False,False,False,False,0,0.494845,0.381443,0.113402,0.000000,0.061856,0.494845,0.381443,0.113402,0.000000,0.061856,0.494845,0.381443,0.113402,0.000000,0.061856,0.494845,0.381443,0.113402,0.000000,0.061856,0.494845,0.381443,0.113402,0.000000,0.061856,0.494845,0.381443,0.113402,0.000000,0.061856
20414,14,笠谷 俊介,ソフトバンク,20202335,2回表,R,日本ハム,False,False,False,0,0,1,False,False,False,False,False,0,0.542553,0.414894,0.138298,0.010638,0.106383,0.542553,0.414894,0.138298,0.010638,0.106383,0.542553,0.414894,0.138298,0.010638,0.106383,0.542553,0.414894,0.138298,0.010638,0.106383,0.542553,0.414894,0.138298,0.010638,0.106383,0.542553,0.414894,0.138298,0.010638,0.106383
20415,15,千賀 滉大,ソフトバンク,20202716,2回裏,R,西武,False,True,False,0,2,2,False,False,False,False,False,0,0.542553,0.414894,0.138298,0.010638,0.106383,0.542553,0.414894,0.138298,0.010638,0.106383,0.542553,0.414894,0.138298,0.010638,0.106383,0.542553,0.414894,0.138298,0.010638,0.106383,0.542553,0.414894,0.138298,0.010638,0.106383,0.542553,0.414894,0.138298,0.010638,0.106383
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54194,33794,漆原 大晟,オリックス,20202513,6回裏,L,ソフトバンク,False,False,False,1,0,2,False,False,False,False,False,0,0.494845,0.381443,0.113402,0.000000,0.061856,0.494845,0.381443,0.113402,0.000000,0.061856,0.494845,0.381443,0.113402,0.000000,0.061856,0.494845,0.381443,0.113402,0.000000,0.061856,0.494845,0.381443,0.113402,0.000000,0.061856,0.494845,0.381443,0.113402,0.000000,0.061856
54200,33800,マルティネス,日本ハム,20202075,4回裏,R,オリックス,True,False,True,1,2,0,False,False,False,False,False,0,0.617978,0.505618,0.179775,0.000000,0.067416,0.600000,0.400000,0.600000,0.000000,0.200000,0.600000,0.200000,0.600000,0.000000,0.200000,0.600000,0.400000,0.600000,0.000000,0.200000,0.600000,0.400000,0.600000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000
54202,33802,上原 健太,日本ハム,20202656,6回裏,R,西武,False,False,False,0,0,1,False,False,False,False,False,0,0.617978,0.505618,0.179775,0.000000,0.067416,0.617978,0.505618,0.179775,0.000000,0.067416,0.617978,0.505618,0.179775,0.000000,0.067416,0.617978,0.505618,0.179775,0.000000,0.067416,0.617978,0.505618,0.179775,0.000000,0.067416,0.617978,0.505618,0.179775,0.000000,0.067416
54204,33804,堀岡 隼人,巨人,20202640,9回表,R,広島,False,False,False,0,0,0,False,False,False,False,False,0,0.505051,0.404040,0.121212,0.000000,0.111111,0.505051,0.404040,0.121212,0.000000,0.111111,0.505051,0.404040,0.1

In [41]:
FTR_btingavg[_FTR5['FTR_pt_btingavg50'].isnull()]

,id,pitcher,pitcherTeam,gameID,inning,batterHand,batterTeam,b1,b2,b3,B,O,S,is_hit0,is_hit1,is_hit2,is_hit3,is_hit4,flg_train,FTR_pt_btingavg00,FTR_pt_btingavg01,FTR_pt_btingavg02,FTR_pt_btingavg03,FTR_pt_btingavg04,FTR_pt_btingavg10,FTR_pt_btingavg11,FTR_pt_btingavg12,FTR_pt_btingavg13,FTR_pt_btingavg14,FTR_pt_btingavg20,FTR_pt_btingavg21,FTR_pt_btingavg22,FTR_pt_btingavg23,FTR_pt_btingavg24,FTR_pt_btingavg30,FTR_pt_btingavg31,FTR_pt_btingavg32,FTR_pt_btingavg33,FTR_pt_btingavg34,FTR_pt_btingavg40,FTR_pt_btingavg41,FTR_pt_btingavg42,FTR_pt_btingavg43,FTR_pt_btingavg44,FTR_pt_btingavg50,FTR_pt_btingavg51,FTR_pt_btingavg52,FTR_pt_btingavg53,FTR_pt_btingavg54
20403,3,クック,ヤクルト,20202650,3回裏,L,中日,True,False,False,0,2,0,False,False,False,False,False,0,0.573034,0.483146,0.191011,0.000000,0.101124,0.573034,0.483146,0.191011,0.000000,0.101124,0.573034,0.483146,0.191011,0.000000,0.101124,0.573034,0.483146,0.191011,0.000000,0.101124,0.573034,0.483146,0.191011,0.000000,0.101124,0.573034,0.483146,0.191011,0.000000,0.101124
20407,7,寺岡 寛治,楽天,20202680,6回表,L,ソフトバンク,False,False,False,1,2,1,False,False,False,False,False,0,0.521739,0.347826,0.152174,0.010870,0.097826,0.521739,0.347826,0.152174,0.010870,0.097826,0.521739,0.347826,0.152174,0.010870,0.097826,0.521739,0.347826,0.152174,0.010870,0.097826,0.521739,0.347826,0.152174,0.010870,0.097826,0.521739,0.347826,0.152174,0.010870,0.097826
20408,8,塩見 貴洋,楽天,20202068,5回裏,L,ソフトバンク,False,False,False,1,2,0,False,False,False,False,False,0,0.521739,0.347826,0.152174,0.010870,0.097826,1.000000,0.600000,0.200000,0.000000,0.400000,0.500000,0.500000,0.250000,0.000000,0.000000,1.000000,0.600000,0.200000,0.000000,0.400000,1.000000,0.600000,0.000000,0.000000,0.400000,1.000000,0.600000,0.200000,0.000000,0.400000
20409,9,齋藤 綱記,オリックス,20202704,3回表,R,楽天,True,False,False,1,1,2,False,False,False,False,False,0,0.494845,0.381443,0.113402,0.000000,0.061856,0.494845,0.381443,0.113402,0.000000,0.061856,0.494845,0.381443,0.113402,0.000000,0.061856,0.494845,0.381443,0.113402,0.000000,0.061856,0.494845,0.381443,0.113402,0.000000,0.061856,0.494845,0.381443,0.113402,0.000000,0.061856
20414,14,笠谷 俊介,ソフトバンク,20202335,2回表,R,日本ハム,False,False,False,0,0,1,False,False,False,False,False,0,0.542553,0.414894,0.138298,0.010638,0.106383,0.542553,0.414894,0.138298,0.010638,0.106383,0.542553,0.414894,0.138298,0.010638,0.106383,0.542553,0.414894,0.138298,0.010638,0.106383,0.542553,0.414894,0.138298,0.010638,0.106383,0.542553,0.414894,0.138298,0.010638,0.106383
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54194,33794,漆原 大晟,オリックス,20202513,6回裏,L,ソフトバンク,False,False,False,1,0,2,False,False,False,False,False,0,0.494845,0.381443,0.113402,0.000000,0.061856,0.494845,0.381443,0.113402,0.000000,0.061856,0.494845,0.381443,0.113402,0.000000,0.061856,0.494845,0.381443,0.113402,0.000000,0.061856,0.494845,0.381443,0.113402,0.000000,0.061856,0.494845,0.381443,0.113402,0.000000,0.061856
54202,33802,上原 健太,日本ハム,20202656,6回裏,R,西武,False,False,False,0,0,1,False,False,False,False,False,0,0.617978,0.505618,0.179775,0.000000,0.067416,0.617978,0.505618,0.179775,0.000000,0.067416,0.617978,0.505618,0.179775,0.000000,0.067416,0.617978,0.505618,0.179775,0.000000,0.067416,0.617978,0.505618,0.179775,0.000000,0.067416,0.617978,0.505618,0.179775,0.000000,0.067416
54204,33804,堀岡 隼人,巨人,20202640,9回表,R,広島,False,False,False,0,0,0,False,False,False,False,False,0,0.505051,0.404040,0.121212,0.000000,0.111111,0.505051,0.404040,0.121212,0.000000,0.111111,0.505051,0.404040,0.121212,0.000000,0.111111,0.505051,0.404040,0.121212,0.000000,0.111111,0.505051,0.404040,0.121212,0.000000,0.111111,0.505051,0.404040,0.121212,0.000000,0.111111
54205,33805,ディプラン,巨人,20202864,7回裏,R,広島,True,False,False,0,0,0,False,False,False,False,False,0,0.505051,0.404040,0.121212,0.000000,0.111111,0.505051,0.404040,0.121212,0.000000,0.111111,0.505051,0.404040,0.121212

In [42]:
# 念のため欠損数の確認
FTR_btingavg.isnull().sum()

id                     0
pitcher                1
pitcherTeam            0
gameID                 0
inning                 0
batterHand           116
batterTeam             0
b1                     0
b2                     0
b3                     0
B                      0
O                      0
S                      0
is_hit0                0
is_hit1                0
is_hit2                0
is_hit3                0
is_hit4                0
flg_train              0
FTR_pt_btingavg00      0
FTR_pt_btingavg01      0
FTR_pt_btingavg02      0
FTR_pt_btingavg03      0
FTR_pt_btingavg04      0
FTR_pt_btingavg10      0
FTR_pt_btingavg11      0
FTR_pt_btingavg12      0
FTR_pt_btingavg13      0
FTR_pt_btingavg14      0
FTR_pt_btingavg20      0
FTR_pt_btingavg21      0
FTR_pt_btingavg22      0
FTR_pt_btingavg23      0
FTR_pt_btingavg24      0
FTR_pt_btingavg30      0
FTR_pt_btingavg31      0
FTR_pt_btingavg32      0
FTR_pt_btingavg33      0
FTR_pt_btingavg34      0
FTR_pt_btingavg40      0


☝：特徴量（FTR_:）に欠損値なし

# データの保存
> ・index, id, 特徴量（FTR_:）のみ

In [43]:
cols = ['id'] + [i for i in FTR_btingavg.columns if i[:3] == 'FTR']
cols

['id',
 'FTR_pt_btingavg00',
 'FTR_pt_btingavg01',
 'FTR_pt_btingavg02',
 'FTR_pt_btingavg03',
 'FTR_pt_btingavg04',
 'FTR_pt_btingavg10',
 'FTR_pt_btingavg11',
 'FTR_pt_btingavg12',
 'FTR_pt_btingavg13',
 'FTR_pt_btingavg14',
 'FTR_pt_btingavg20',
 'FTR_pt_btingavg21',
 'FTR_pt_btingavg22',
 'FTR_pt_btingavg23',
 'FTR_pt_btingavg24',
 'FTR_pt_btingavg30',
 'FTR_pt_btingavg31',
 'FTR_pt_btingavg32',
 'FTR_pt_btingavg33',
 'FTR_pt_btingavg34',
 'FTR_pt_btingavg40',
 'FTR_pt_btingavg41',
 'FTR_pt_btingavg42',
 'FTR_pt_btingavg43',
 'FTR_pt_btingavg44',
 'FTR_pt_btingavg50',
 'FTR_pt_btingavg51',
 'FTR_pt_btingavg52',
 'FTR_pt_btingavg53',
 'FTR_pt_btingavg54']

In [44]:
FTR_btingavg[cols].to_pickle(data_path / ("%s_FTR_pt_btingavg.pkl" % prefix))